In [4]:
import requests
import json
import pandas as pd



*   API queries below
*   api_key and file path are both placeholders, input your key & path
*   2010-2023 and 2024 queries are seperated because the variable name for MBS area, region, and state code changes.



In [ ]:
# Define API key and base URL
api_key = "0cbe74e555fe4b3dac32591deacb7248f4dafae0"
base_url = "https://api.census.gov/data"

# Define variables of interest 2010-2023
variables = [
    "GTCBSA",      # Metropolitan Core Based Statistical Area FIPS Code
    "PELAYFTO",    # Labor Force-(layoff) from full-time job
    "PELAYDUR",    # Labor Force-(layoff) # weeks looking for job
    "HEFAMINC",    # Household-total family income
    "PESEX",       # Demographics-sex
    "PTDTRACE",    # Demographics-race
    "PEEDUCA",     # Demographics-highest level of school completed
    "PEMARITL",    # Demographics-marital status
    "HRHTYPE",     # Household-type of family/single individual
    "HRNUMHOU",    # Household-total # of members
    "GEREG",       # REGION
    "GESTFIPS"     # FIPS STATE Code
]

# Define year range (2010-2023)
years = range(2010, 2024)

# Define months
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

# Query data 2010-2023
for year in years:
    for month in months:
        # Construct the API request URL for each year in year range, month in month range
        url = f"{base_url}/{year}/cps/basic/{month}?get={','.join(variables)}&for=state:*&key={api_key}"

        # Query data from Census API
        response = requests.get(url)

        # Confirm that the query was successful
        if response.status_code == 200:
            # Convert response to DataFrame
            data = response.json()
            df = pd.DataFrame(data[1:], columns=data[0])
            df.to_csv(f"/content/test/cps_data_{year}_{month}.csv")
            print(f"Data for {year}, {month} fetched")
        else:
            print(f"Failed to retrieve data for {year}, {month}: {response.status_code}")

In [ ]:
# Define varibles of interest for 2024
variables_2024 = [
    "CBSA",         # Metropolitan Core Based Statistical Area FIPS Code
    "PELAYFTO",     # Labor Force-(layoff) from full-time job
    "PELAYDUR",     # Labor Force-(layoff) # weeks looking for job
    "HEFAMINC",     # Household-total family income
    "PESEX",        # Demographics-sex
    "PTDTRACE",     # Demographics-race
    "PEEDUCA",      # Demographics-highest level of school completed
    "PEMARITL",     # Demographics-marital status
    "HRHTYPE",      # Household-type of family/single individual
    "HRNUMHOU",     # Household-total # of members
    "REGION",       # REGION
    "STATE"         # FIPS STATE Code
]

# Define months for 2024
months_2024 = ['jan', 'feb', 'mar']

# Query data for 2024
for month in months_2024:
    # Construct the API request URL for each months in the 2024 month range
    url_2024 = f"{base_url}/2024/cps/basic/{month}?get={','.join(variables_2024)}&for=state:*&key={api_key}"

    # Query data from Census API
    response_2024 = requests.get(url_2024)

    # Confirm that the query was successful
    if response_2024.status_code == 200:
        # Convert response to DataFrame
        data_2024 = response_2024.json()
        df_2024 = pd.DataFrame(data_2024[1:], columns=data_2024[0])

        # Rename DF column headings to match the bulk of the data
        df_2024.rename(columns={"CBSA": "GTCBSA", "REGION": "GEREG", "STATS": "GESTFIPS"}, inplace=True)
        df.to_csv(f"/content/drive/MyDrive/CPSDataYEARmo/cps_data_2024_{month}.csv")
        print(f"Data for 2024, {month} fetched")
    else:
        print(f"Failed to retrieve data for 2024, {month}: {response_2024.status_code}")

Ideally, one could directly send the query outputs to a dataframe and work with just the one file, but I was having issues w/ virtual machine memory. See below to merge each downloaded csv from your filepath into one dataframe.

In [22]:
import re

# Redefine years, months & MTD 2024
years = range(2010, 2024)
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']
months_2024 = ['jan', 'feb', 'mar']

# Empty placeholder list to store file paths
file_paths = []

# Generate each file path name & input to a list
for year in years:
    for month in months:
        file_paths.append(f"/content/drive/MyDrive/CPSDataYEARmo/cps_data_{year}_{month}.csv")

for month in months_2024:
  file_paths.append(f"/content/drive/MyDrive/CPSDataYEARmo/cps_data_2024_{month}.csv")

# Empty placeholder list to list of dfs for each csv queried
dfs = []

for file_path in file_paths:
    # Read each csv file into a df
    df = pd.read_csv(file_path)

    # Use regex to extract the year and month for each csv file path to differentiate in the final list of dfs
    match = re.search(r'\d{4}_(\w+).csv', file_path)
    if match:
        month = match.group(1)
        year = file_path.split('_')[2]

        # Add a new column 'month_year' with the format 'Mon-YEAR'
        df['month_year'] = f"{month.capitalize()}-{year}"

        # Store the generated df into the dfs list & loop
        dfs.append(df)

# Concatenate the list of dfs into one dataframe
combined_df = pd.concat(dfs, ignore_index=True)

# Download the combined dataframe
combined_df.to_csv("/content/drive/MyDrive/CombinedCPSData/comb_cps_data.csv")